# Multi-Chainage

Les éléments exécutables peuvent facilement être utilisés pour enchaîner plusieurs chaînes.

In [10]:
#%pip install –quiet langchain langchain-openai
OPENAI_API_KEY= "OPENAI_API_KEY"

In [11]:
import os
os.environ['OPENAI_API_KEY']= OPENAI_API_KEY

In [12]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("De quelle ville vient {person}?")
prompt2 = ChatPromptTemplate.from_template(
    "De quelle pays vient la ville {city}? Réponds en {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "tagalog"})

'Ang lungsod ni Barack Obama ay galing sa Chicago, sa estado ng Illinois, sa Estados Unidos.'

In [14]:
type(prompt1)

langchain_core.prompts.chat.ChatPromptTemplate

In [18]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "Génère une couleur {attribute}. Retourne le nom de la couleur et rien d'autre:"
)
prompt2 = ChatPromptTemplate.from_template(
    "Quel est le fruit de la couleur: {color}. Retourne le nom du fruit et rien d'autre:"
)
prompt3 = ChatPromptTemplate.from_template(
    "Quel est le pays dont le drapeau porte la couleur: {color}. Retourne le nom du pays et rien d'autre:"
)
prompt4 = ChatPromptTemplate.from_template(
    "Quel est la couleur de {fruit} et du drapeau de {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4 
)

In [19]:
question_generator.invoke("diplomatique")

'La couleur de la poire est vert et le drapeau de Bahreïn est rouge et blanc avec un triangle noir.'

### Branchement et fusion de chaines

Il se peut que vous souhaitiez que la sortie d'un composant soit traitée par deux autres composants ou plus. RunnableParallels vous permet de diviser ou de forker la chaîne afin que plusieurs composants puissent traiter l'entrée en parallèle. Par la suite, d'autres composants peuvent joindre ou fusionner les résultats pour synthétiser une réponse finale. Ce type de chaîne crée un graphe de calcul qui ressemble à ce qui suit:
```     
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
    Combine
```

In [6]:
planner = (
    ChatPromptTemplate.from_template("Génère-moi une dissertation sur: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "Liste les aspects positifs ou 'pros' sur {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "Liste les aspects négatifs ou 'cons' {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Génère une conclusion en fonction de la critique reçu"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [7]:
texte = chain.invoke({"input": "Le déploiement de l'IA dans les SI"})
print(texte.replace(".", ".\n"))

En conclusion, le déploiement de l'intelligence artificielle (IA) dans les systèmes d'information (SI) présente à la fois des avantages significatifs et des défis importants à relever.
 Les aspects positifs tels que l'optimisation des processus métier, l'amélioration de la productivité, la personnalisation de l'expérience client, la prévention des risques et l'adaptabilité aux changements du marché offrent des opportunités considérables pour les entreprises.


Cependant, il est crucial de ne pas sous-estimer les conséquences des défis identifiés, notamment en ce qui concerne la sécurité des données sensibles, la formation des collaborateurs, le contrôle de l'IA, les coûts impliqués, l'impact sur l'emploi, les risques de biais et de discrimination, la complexité de l'intégration et la conformité aux réglementations.


En somme, il est primordial pour les entreprises d'adopter une approche équilibrée, en maximisant les avantages de l'IA tout en gérant efficacement les risques et enjeux a

> En conclusion, le déploiement de l'intelligence artificielle dans les systèmes d'information offre de nombreux avantages en termes d'automatisation des tâches, d'analyse des données en temps réel, de prédiction des tendances et d'amélioration de la performance des SI. Cependant, il est crucial de prendre en compte les risques liés à la protection de la vie privée, à la sécurité informatique et aux enjeux éthiques. Il est donc impératif de mettre en place des politiques de gouvernance des données, de former les utilisateurs, de sensibiliser les entreprises aux enjeux éthiques et de promouvoir une approche responsable de l'IA pour garantir que ses bénéfices profitent à l'ensemble de la société.



## Parallelisme

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("écris-moi une blague sur {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("écrit un poème de deux lignes sur {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "la montagne"})

{'joke': AIMessage(content="Pourquoi les montagnes sont-elles si drôles ? Parce qu'elles ont toujours une crête de quoi rire !"),
 'poem': AIMessage(content='La montagne majestueuse, couronnée de neige éternelle, garde les secrets du monde dans ses crevasses profondes.')}